In [11]:
import os
from multiprocessing import Pool


def safe_readline(f):
    pos = f.tell()
    while True:
        try:
            return f.readline()
        except UnicodeDecodeError:
            pos -= 1
            f.seek(pos)
            
            
def async_kd_tokenizer(filename, worker_id, num_workers):
    with open(filename, 'r') as f:
        size = os.fstat(f.fileno()).st_size
        print(f'size {size}')
        chunk_size = size // num_workers
        offset = worker_id * chunk_size
        end = offset + chunk_size
        f.seek(offset)
        print(f'offset {offset}')
        if offset > 0:
            safe_readline(f)    # drop first incomplete line
        lines = []
        line = f.readline()
        i = 0
        while line:
            line = line.replace(" ", '').replace("\n", '')
            if not line:
                line = f.readline()
                continue
            lines.append(line)
            if f.tell() > end:
                break
            line = f.readline()
            i += 1
        return lines
    
    
def encode_file(path, workers=4):
    assert os.path.exists(path)
    results = []
    workers_thread = []
    pool = Pool(processes=workers)
    for i in range(workers):
        w = pool.apply_async(
            async_kd_tokenizer,
            (path, i, workers),
        )
        workers_thread.append(w)
    pool.close()
    pool.join()

    for w in workers_thread:
        result = w.get()    # lsp
        results += result
        
    return results

results = encode_file('/Users/lisen/Downloads/如果蜗牛有爱情.txt')
# results = encode_file('/Users/lisen/Downloads/5236.txt')

with open('如果蜗牛有爱情result.txt', 'w', encoding='utf-8') as fw:
    for line in results:
        fw.write(line)

size 959560
size 959560
size 959560
size 959560
offset 239890
offset 0
offset 719670
offset 479780


7243